In [ ]:
#competition.m

def competition(oldPop,A,B,C):
    totalPop = sum(oldPop)
    newPop = newPop = A*oldPop/(1+B+C*totalPop)

    return newPop

In [ ]:
# deeptill.m

def deepTill(upperBank,lowerBank,tillFrequency,currentYear):
    if tillFrequency(currentYear) == 1:
        tmp = upperBank
        upperBank = lowerBank
        lowerBank = tmp

    return upperBank, lowerBank

In [ ]:
# extinction.m

import random
from math import inf
import numpy as np


def extinction(oldPopulation,fieldSize):
    if fieldSize == inf:
        newPopulation = oldPopulation

    elif fieldSize == 0:
        raise "old pop returned"

    else:
        # %         rng('shuffle');
        nGeno = len(oldPopulation);
        rndExtinct = random.random(1,nGeno);
        newPopulation = np.zeros(1,nGeno);

        for j in range(nGeno):
            if oldPopulation[j] == 0:
                # %first check if jth genotype population is 0
                newPopulation[j] = 0
            elif oldPopulation[j] >= 1/fieldSize:
                # %next check if jth genotype number in field >=1
                newPopulation[j] = oldPopulation(j)
            elif rndExtinct[j] > 0.5:
                # %jth genotype survives extinction!
                newPopulation[j] = 1/fieldSize
            else:
                # %otherwise, jth genotype dies...
                newPopulation[j] = 0

    return newPopulation

In [ ]:
#getresistancetime

def getResistanceTime(time,res_frac,f,varargin):
    # %time is a (Mx1) array. res_frac is an (Mx1) array. time and res_frac
    # %must have the same number of rows, otherwise an error is returned. f
    # %is the resistant fraction whose time-to-resistance will be found. f
    # %may be a (Px1) array. Function returns the time-to-resistance, tR,
    # %which is a (Px1) array. varargin is a Px1 array which may be used to
    # %keep "count" of how many times resistance developed for each element
    # %of f. If resistance is found for current value of f, a counter
    # %variable is incremented by 1. This is then returned in counter. This
    # %is useful for counting how many times resistance developed over many
    # %function calls
    
    if len(time) != len(res_frac):
        raise 'time and res_frac must have equal number of rows'
        
    else:
        # %Counter variable
        if varargin:
            counter = varargin[1];
        else:
            counter = np.zeros(len(f),1)

        
        # %number of evaluation points, P
        P = len(f)
        
        # %output variable, a (Px1) array
        tR = np.zeros(P,1)
        
        # %Loop through f and find corresponding times.
        for j in range(len(P)):

            tmp = find(res_frac>f(j),1,'first')

            if tmp == 1:
                # %The initial resistance exceeded the threshold
                tR[j] = 0;
                
                # %Increment counter
                counter[j] = counter[j] + 1
            elif tmp:
                # %If the threshold is exceeded, interpolate to find the time

                # %Temporary time and resistant fraction variables for
                # %readability
                t2 = time(tmp)
                t1 = time(tmp-1)
                f2 = res_frac(tmp)
                f1 = res_frac(tmp-1)

                # %Linear interpolation
                tR[j] = (t2 - t1) / (f2 - f1) * (f[j] - f1) + t1
                
                # %Increment counter
                counter[j] += 1
            else:
                # %The threshold was never exceeded
                tR[j] = inf
                


In [ ]:
def mating(population,s,key):
    nLoci,nGeno = size(key);


# %% Pure outcrossing
    if sum(population) == 0:
        # raise('Population is zero')
        newFracOut = np.zeros(1,len(population))
        
    else:
        nLoci = size(key,1);
        nGeno = size(key,2);
        newFracOut = ones(1,nGeno);

        p = np.zeros(1,nLoci);

        %determine current allele frequency
        for j in range(nLoci):
            for k in range(nGeno):
                if key(j,k) == 2:
                    p[j] = p[j] + 2*population[k]
                elif key[j,k] == 1:
                    p[j] = p[j] + population[k]

        p = 0.5*p/sum(population)
        q = 1 - p

        # %determine new genotype fraction
        for j in range(nLoci):
            for k in range(nGeno):
                if key(j,k) == 2:
                    newFracOut[k] = newFracOut(k)*p(j)^2
                elif key(j,k) == 1:
                    newFracOut[k] = newFracOut(k)*2*p(j)*q(j)
                else:
                    newFracOut[k] = newFracOut(k)*q(j)^2

    
# %% Pure selfing
selfingTable = dlmread(strcat('selfTable',num2str(nLoci),'.txt'));
newFracSelf = population*selfingTable;

# %% Total new fraction
newFrac = s*newFracSelf + (1-s)*newFracOut;

In [ ]:
function mutatedPopulation = mutation(population,muS2R,muR2S,targetLocus,key)

mu = muR2S;
nu = muS2R;

# %define a new population array and initilize it to the current population
mutatedPopulation = population;

# %cycle through genotypes
for j = 1:size(key,2)
    # %Note: Since mutation is taken one locus at a time, each
    # %genotype can only become 2 other genotypes and vice versa, i.e.
    # %for locus 1, [1,2,1] -> [0,2,1] or [2,2,1] only. For each
    # %genotype, we need to first locate the other two that it can
    # %interact with
    tmp = mod(j,3^(targetLocus-1))+1;

    indSS = tmp;
    indRS = indSS + 3^(targetLocus-1);
    indRR = indRS + 3^(targetLocus-1);

    if key(targetLocus,j) == 2
        # %if the current genotype + locus has 2 resistant alleles
        mutatedPopulation(indRR) = mutatedPopulation(indRR) - 2*mu*population(indRR) + mu^2*population(indRR);
        mutatedPopulation(indRS) = mutatedPopulation(indRS) + 2*mu*population(indRR) - 2*mu^2*population(indRR);
        mutatedPopulation(indSS) = mutatedPopulation(indSS) + mu^2*population(indRR);
    elseif key(targetLocus,j) == 1
        # %if the current genotype + locus has 1 resistant alleles
        mutatedPopulation(indRR) = mutatedPopulation(indRR) + nu*population(indRS);
        mutatedPopulation(indRS) = mutatedPopulation(indRS) - nu*population(indRS) - mu*population(indRS);
        mutatedPopulation(indSS) = mutatedPopulation(indSS) + mu*population(indRS);    
    else
        # %if the current genotype + locus has 0 resistant alleles
        mutatedPopulation(indRR) = mutatedPopulation(indRR) + nu^2*population(indSS);
        mutatedPopulation(indRS) = mutatedPopulation(indRS) + 2*nu*population(indSS) - 2*nu^2*population(indSS);
        mutatedPopulation(indSS) = mutatedPopulation(indSS) - 2*nu*population(indSS) + nu^2*population(indSS);  
    end

    population = mutatedPopulation;

end


In [ ]:
function plt_han = plotPopulation(LifeStage,key,varargin)
# %Accepts a lifestage of the population structure and plots the population
# %versus time. Required inputs are LifeStage (e.g. Population.germination)
# %and the genotype key (e.g. Params.key). Optional argument is the plot
# %type: 
# %'total' = total population density 
# %'totalR' = total population %possessing resistant allele (including
# %heterozygotes) 
# %'totalRR' = total %population possessing 2 resistant alleles 
# %'fracR' = fraction of population containing resistant allele (including
# %heterozygotes)
# %'fracRR' = fraction of population containing 2 resistant alleles

if isempty(varargin)
    keyword = 'total';
else
    keyword = varargin{1};
end

[nYears,nGeno,nCohorts] = size(LifeStage);
N = nYears*nCohorts;

newPop = zeros(N,nGeno);

k = 1;
for y = 1:nYears
    for c = 1:nCohorts
        newPop(k,:) = LifeStage(y,:,c);
        k = k + 1;
    end
end

switch keyword
    case 'total'
        pltPop = sum(newPop,2);
        time = 0:N-1;
        time = time / nCohorts;
        ylab = 'Population, ($pl/m^2$)';
        ymax = Inf;
    case 'totalR'
        pltPop = sumResistant(newPop,key);
        time = 0:N-1;
        time = time / nCohorts;
        ylab = 'Population, ($pl/m^2$)';
        ymax = Inf;
    case 'totalRR'
        pltPop = sumResistant(newPop,key,'homozygous');
        time = 0:N-1;
        time = time / nCohorts;
        ylab = 'Population, ($pl/m^2$)';
        ymax = Inf;
    case 'fracR'
        tmp = sumResistant(newPop,key);
        pltPop = tmp./sum(newPop,2);
        time = 0:N-1;
        time = time / nCohorts;
        ylab = 'Fraction resistant';
        ymax = 1;
    case 'yearlyfracR'
        newPop = newPop(nCohorts:nCohorts:end,:);
        tmp = sumResistant(newPop,key);
        pltPop = tmp./sum(newPop,2);
        time = 0:nYears-1;
        ylab = 'Fraction resistant';
        ymax = 1;
        
end

hold all
plt_han = plot(time,pltPop,'.-');
xlabel('Year','FontSize',20,'Interpreter','latex')
ylabel(ylab,'FontSize',20,'Interpreter','latex')
axis([0 max(time) 0 ymax])
set(gca,'FontSize',15)
hold off

In [ ]:
def prepareParams_2(Params):
# %Takes the user defined parameter structure array created in readData and 
# %prepares three variables for the simulation: UpperSeedBank, LowerSeedBank 
# %and Params. UpperSeedBank and LowerSeedBank contain the initial population
# %information.

# %% General parameters

    P.General = Params.General;
    P.Init = Params.Init;
    P.Cultivation = Params.Cultivation;
    P.Germination = Params.Germination;
    P.Hand = Params.Hand;
    P.Mature = Params.Mature;
    P.SeedProd = Params.SeedProd;
    P.Mutation = Params.Mutation;
    P.Predation = Params.Predation;
    P.Winter = Params.Winter;
    P.Herbicide1 = Params.Herbicide1;
    P.Herbicide2 = Params.Herbicide2;
    P.Herbicide3 = Params.Herbicide3;
    P.Herbicide4 = Params.Herbicide4;

# %Define a few of the general parameters locally for use in
# %the function (not most computationally efficient, but improves
# %readability)
nLoci = Params.General.nLoci;

# %Generate a key for relating each genotype to the number of resistant
# %alleles it posseses (0,1, or 2) at each gene locus. (Subfunction contained
# %in this file).
[P.key,P.keyText] = generateKey(nLoci);


# %% Initialize the upper seed bank

# %Calculate the fraction of each genotype by calling the matingEquilibrium
# %function (subfunction in this file).
tmp = matingEquilibrium(P.Init.Upper.ResAlleleFreq,P.General.selfingCoeff,P.key);

# %Populate UpperSeedBank 
P.General.UpperSeedBank = tmp*Params.Init.Upper.seedDensity;

# %% Initialize the lower seed bank

# %Calculate the fraction of each genotype by calling the matingEquilibrium
# %function (subfunction in this file).
tmp = matingEquilibrium(P.Init.Lower.ResAlleleFreq,P.General.selfingCoeff,P.key);

# %Populate LowerSeedBank 
P.General.LowerSeedBank = tmp*Params.Init.Lower.seedDensity;



# %% Subfunctions
def generateKey(nLoci):
# %create a "key" array with rows correpsonding to a single gene locus,
# %columns corresponding to unique genotypes, and each element containing
# %the number of resistant alleles that a genotype has at a given gene
# %locus. Example: key = [  0,0,0,1,1,1,2,2,2 ;
# %   0,1,2,0,1,2,0,1,2 ] corresponds to the 9 unique genotypes when 2
# %   loci are considered. First column contains 0 resistant alleles at
# %   loci 1 and loci 2.
# %This function also creates a cell array which contains the same
# %information in more readable text format. This way, the user can
# %easily correlate columns in variable "key" to a genotype. Example:
# %key_text = [S1S1S2S2, S1S1R2S2, S1S1R2R2, ...etc, just run the code to
# %see the output.

# %number of unique genotypes
nGeno = 3^nLoci;

# %initialize key array
key = zeros(nLoci,nGeno);

# %populate the key array using the function odometer
tmp = zeros(nLoci,1);
for k = 1:nGeno
    key(:,k) = tmp;
    tmp = odometer(tmp,2);

key_text = 0;

# %create the text version for easier viewing by user
key_text = cell(1,nGeno)
for k = 1:nGeno
    tmp = ''
    for j = 1:nLoci:
        locus_string = num2str(j)
        if key(j,k) == 0:
            tmp = strcat(tmp,'S',locus_string,'S',locus_string)
        elseif key(j,k) == 1
            tmp = strcat(tmp,'R',locus_string,'S',locus_string)
        else
            tmp = strcat(tmp,'R',locus_string,'R',locus_string)



    key_text{k} = tmp


    
def odometer(odo,base_num):
# %% Odometer function
# %This function counts using a user specified number base. A digit takes
# %on integer values between 0 and base_num. This function increments the
# %right-most digit, unless the right-most digit is at the maximum value,
# %in which case it is set to the first element of base_num and the next
# %element is incremented. For example: odo = [0,0,1] will become odo =
# %[0,0,2]. A second function call on odo will produce odo = [0,1,0].
# %This works like a standard car odometer, but with an arbitrary number
# %system.

    N = length(base_num);       %number of possible elements
    Ndigits = length(odo);      %number of elements in odo

    j = 1;
    while j <= Ndigits
        if odo(j) < base_num
            odo(j) = odo(j)+1
            break

        odo(j) = 0
        j = j + 1


def  matingEquilibrium(p,selfCoeff,key):
    q = 1-p;
    nLoci = size(key,1);
    nGeno = size(key,2);
    populationFraction_outCross = ones(1,nGeno);
    %     populationFraction_selfing = ones(1,nGeno);

    for j = 1:nLoci
        for k = 1:nGeno
            if key(j,k) == 2
                populationFraction_outCross(k) = populationFraction_outCross(k)*p(j)^2;
            elif key(j,k) == 1
                populationFraction_outCross(k) = populationFraction_outCross(k)*2*p(j)*q(j);
            else:
                populationFraction_outCross(k) = populationFraction_outCross(k)*q(j)^2;


In [ ]:
function p = readData( )
# %Reads data from text files created by Excel and stores them into a
# %structure for the program to use.

format long g;

# %% Create struct to hold all the data.
p = struct('General',{},'Init', {},'Germination', {}, 'Cultivation', {}, ...
    'Hand', {}, 'Mature', {},'SeedProd', {},'Mutation', {}, 'Predation', {}, ...
    'Winter', {}, 'Herbicide1', {}, 'Herbicide2', {}, 'Herbicide3', {}, 'Herbicide4', {});

# %% Initiate static parameters.
p(1).General.currentYear = 1;
p(1).General.currentCohort = 1;

# %% Get Windows user name to construct correct file paths
username = getenv('USERNAME');

# %% Read and store life cycle parameter data.
path = strcat('C:\Users\', username, '\Desktop\Weed_Resistance_Data.txt');

# %Read file to local variable
params_in = dlmread(path);

p(1).General.nSim = params_in(1,1);                # %number of simulations
p(1).General.nYears = params_in(2,1);              # %number of years
p(1).General.nCohorts = params_in(3,1);             #%number of cohorts
p(1).General.nLoci = 4;                             #%number of gene loci
p(1).General.nGeno = 3^p.General.nLoci;            # %number of genotypes
p(1).General.fieldSize = params_in(5,1);           # %field size
p(1).General.selfingCoeff = params_in(6,1);        # %selfing coefficient 
p(1).General.seedDelay = params_in(7,1);           # %seed delay
p(1).General.femaleFrac = params_in(8,1);          # %fraction female
p(1).Init.Upper.seedDensity = params_in(9,1);      # %initial upper bank density
p(1).Init.Lower.seedDensity = params_in(10,1);      # %initial lower bank density

p(1).Init.Upper.ResAlleleFreq = params_in( 11:2:17,1 );  # %initial upper resistant allele frequency
p(1).Init.Lower.ResAlleleFreq = params_in( 12:2:18,1 );   #%initial lower resistant allele frequency

p(1).General.f_RO = params_in(19,1);         # %latent time
p(1).General.f_CR = params_in(20,1);    # %critical response
p(1).General.f_TR = params_in(21,1);    # %complete resistance time

row = 22;
for i = 1:p.General.nYears
    for j = 1:p.General.nCohorts
        p(1).Germination.survivalFraction(i,j) = params_in(row, 1);
        p(1).Cultivation.survivalFraction(i,j) = params_in(row, 2);
        p(1).Hand.survivalFraction(i,j) = params_in(row, 3);
        p(1).Mature.maxPlants(i,j) = params_in(row, 4);
        p(1).Mature.cropParam(i,j) = params_in(row, 5);
        p(1).Mature.weedCompetition(i,j) = params_in(row, 6);
        p(1).SeedProd.maxYield(i,j) = params_in(row, 7);
        p(1).SeedProd.cropParam(i,j) = params_in(row, 8);
        p(1).SeedProd.weedCompetition(i,j) = params_in(row, 9);
        p(1).Mutation.muR2S_locus1(i,j) = params_in(row, 10);
        p(1).Mutation.muS2R_locus1(i,j) = params_in(row, 11);
        p(1).Mutation.muR2S_locus2(i,j) = params_in(row, 12);
        p(1).Mutation.muS2R_locus2(i,j) = params_in(row, 13);
        p(1).Mutation.muR2S_locus3(i,j) = params_in(row, 14);
        p(1).Mutation.muS2R_locus3(i,j) = params_in(row, 15);
        p(1).Mutation.muR2S_locus4(i,j) = params_in(row, 16);
        p(1).Mutation.muS2R_locus4(i,j) = params_in(row, 17);
        p(1).Predation.survivalFraction(i,j) = params_in(row, 18);
        p(1).Winter.uppersurvivalFraction(i,j) = params_in(row, 19);
        p(1).Winter.lowersurvivalFraction(i,j) = params_in(row, 20);
        row = row + 1;



# %% Read and store tilling data.
path = strcat('C:\Users\', username, '\Desktop\Tilling_Data.txt');
p(1).General.TillingFreq = dlmread(path);

# %% Read and store herbicide data.
path = strcat('C:\Users\', username, '\Desktop\Herbicide_Data.txt');
data = dlmread(path);

data_rows = size(data, 1);
row = 1;
for i = {'Herbicide1', 'Herbicide2', 'Herbicide3', 'Herbicide4'}
    if row > data_rows
        break
    end
    p(1).(i{1}).targetLocus = data(row, 1);
    for k = 1:p.General.nYears
        for m = 1:p.General.nCohorts
            p(1).(i{1}).efficacySS(k, m) = data(row, 2);
            p(1).(i{1}).efficacyRS(k, m) = data(row,3);
            p(1).(i{1}).efficacyRR(k, m) = data(row,4);
            row = row + 1;


In [ ]:
def [int_risk,res_risk,res_data,RoverT] = resistanceRisk(nSim)

#Runs the Main weed resistance code nSim times, determining the resistance
#risk. The time to resistance is found for every value specified in p_eval,
#which must be numbers between 0 and 1 (time is linearly interpolated
#between years). The def returns the population for each of nSim
#simulations in res_frac, an array of resistance propability co


## Run simulation nSim times

#Life stage to consider (seed bank)
lifestage = 'seedBank' 

for n = 1:nSim
    
    #Call Main resistance def.
    #(Params stays the same for all sims, so only store it once)
    if n == 1
        [Population,Params] = Main_2() 
        RoverT = zeros(Params.General.nYears,nSim) 
        res_data = zeros(3,nSim) 
        res_risk = zeros(Params.General.nYears,1) 
        years = 0:Params.General.nYears-1 
        
        
    else
        [Population,~] = Main_2(Params) 
        
     
    
    #Count the resistants in each year, and the total population in each
    #year.
    
    #resistant population
    res_pop = sumResistant(Population.(lifestage)(:,:, ),Params.key) 
    
    #total population
    tot_pop = sum(Population.(lifestage)(:,:, ),2) 
    
    #resistant fraction stored in res_frac
    RoverT(:,n) = res_pop./tot_pop 
    
    #Store resistance metrics: t_latent, t_crit, and t_total
    res_data(:,n) = Population.Res 
    
    #Find the resistance risk as a def of time
    ind = find( RoverT(:,n) >= Params.General.criticalResTime ) 
    res_risk(ind) = res_risk(ind) + 1 
    
    
    
 

#Convert res_risk to a fraction
res_risk = res_risk / nSim 

#Find integrated res_risk
int_risk = trapz(years,res_risk)/length(years) 

#Plot the results
scrz = get(0,'ScreenSize') 
figure('Position',[scrz(1)+100,scrz(2)+100,scrz(3)*0.6,scrz(4)*0.6])

#All simulations
ax1 = axes('Position',[0.1,0.1,0.35,0.85]) 
hold all
plot(years,RoverT) 
axis([0,years( ),0,1])

#Resistance emergence
ax1 = axes('Position',[0.6,0.1,0.35,0.85]) 
t = linspace(years(1),years( ),1000) 
plot(t,pchip(years,res_risk*100,t),'k','LineWidth',2) 
axis([0,years( ),0,100])

 

## Subdefs
def Rpop = sumResistant(pop,key)

    #Determine number of gene loci and genotypes from key variable
    [nLoci,nGeno] = size(key) 
    
    #Determine number of years and cohorts from population variable
    [nYears,~,nCohorts] = size(pop) 
    
    #Initialize output array to 0
    Rpop = zeros(nYears,nCohorts) 
    
    #Sum all with resistant allele
    for G = 1:nGeno
        for L = 1:nLoci
            if key(L,G) == 2 || key(L,G) == 1
                Rpop(:,:) = Rpop(:,:) + squeeze(pop(:,G,:)) 
                break
             
         
     
    
 


In [ ]:
function [ resdata ] = resMetrics( resdata, currentYear, seedPop, ...
                                        key, onset, critical, complete )
                                    
%resMetrics updates weed resistance data using population data from the 
%current year of the simulation. It takes as parameters the resistance
%metrics array, the current year, the mature population for the current
%year, the key, and the user-defined threshold values (percentages at which
%weed resistance is considered to have developed, reached a critical value,
%and affected the whole population).

R = sumResistant(seedPop(currentYear, :, end),key); %resistant population
T = sum(seedPop(currentYear, :, end),2);    %total population (at end of year)

%Checks if: 1) the fraction of the populattion that is resistant has exceeded 
%the onset value and 2) the time of resistance onset has not been set. If
%both conditions are met, the time of resistance onset is set to the
%current year.
if R(end)/T(end) >= onset && resdata(1, 1) == 0
    resdata(1, 1) = currentYear;
end

%Checks if: 1) the fraction of the populattion that is resistant has exceeded 
%the critical value and 2) the time of critical resistance has not been set. 
%If both conditions are met, the time of critical resistance is set to the
%current year.
if R(end)/T(end) >= critical && resdata(1, 2) == 0
    resdata(1, 2) = currentYear;
end

%Checks if: 1) the majority of the population is resistant (according to 
%the value specified by the user) and 2) the time of complete resistance has
%not been set. If both conditions are met, the time of complete resistance 
%is set to the current year.
if R(end)/T(end) >= complete && resdata(1, 3) == 0
    resdata(1, 3) = currentYear;
end

end



In [ ]:
function [metrics,data_output] = runSimulation(varargin)
%% Initialization

    %Create a program status bar 
    h = waitbar(0, 'Please wait', 'Name', 'Plot progress', ...
    'CreateCancelBtn', 'setappdata(gcbf, ''canceling'', 1)');
    setappdata(h, 'canceling', 0)


    %Run Main function with no arguments once to get the params variable
    %populated
    if ~isempty(varargin)
        Params = varargin{1};
    else
        tmp = readData();
        Params = prepareParams_2(tmp);
    end
    
    %Number of simulations to run
    if isinf(Params.General.fieldSize)
        nSim = 1;
    else
        nSim = Params.General.nSim;
    end
    
    nYears = Params.General.nYears;
    nCohorts = Params.General.nCohorts;
    
    %Frequency values to find the corresponding resistance time for.
    %Includes three user defined values, as well as evenly spaced
    %values from 0.05 to 0.95 fraction resistant.
    f_RO = Params.General.f_RO;
    f_CR = Params.General.f_CR;
    f_TR = Params.General.f_TR;
    f = 0.05:0.05:0.95;
    f = [f_RO;f_CR;f_TR;0.01;f';0.99];
    f_len = length(f);
    
    counter = zeros(f_len,1);
    
    %Variables to store results
    %variable to locally store the seed bank population
    data_store = zeros(nCohorts*nYears,nSim+1);
    
    %variable for sending to the excel file (includes upper and lower seed
    %banks). NOTE: I retroactively added this variable. It probably would
    %be better code to have data_store and data_output be combined into a
    %single array.
    data_output = struct('upperbank_freq',zeros(nCohorts*nYears,nSim+1),...
                         'lowerbank_freq',zeros(nYears,nSim+1));
    
    %structure array for hold population metrics
    metrics = struct('IRF',zeros(nSim,1),'IRF_stats',0,...
                                         'probs',zeros(f_len,1),...
                                         't2R',Inf(f_len,nSim),...
                                         't2R_stats',Inf(f_len,5));
    
    %Make first column contain time values
    time_step = 1/nCohorts;
    data_store(:,1) = 0:time_step:nYears-time_step;
    data_output.upperbank_freq(:,1) = 0:time_step:nYears-time_step;
    data_output.lowerbank_freq(:,1) = 0:nYears-1;
    
    %% Perform nSim simulations, storing the results 
    for n = 1:nSim
        
        %Call main function
        [ Population,~ ] = Main(Params);
       
        %% All data for direct export
        %Sum over all genotypes to find the total resistant and total seed
        %population for each cohort/year
        R = sumResistant( Population.seedBank,Params.key );
        T = squeeze( sum(Population.seedBank,2) );
        R_lower = sumResistant( Population.lowerBank,Params.key );
        T_lower = squeeze( sum(Population.lowerBank,2) );
        
        %Calculate fraction resistant for each year/cohort
        f_res = R./T;
        f_res_lower = (R_lower ./ T_lower);
        
        %Zero divided by zero errors produce NaN for f_res. Replace NaN by
        %0, since NaN errors come when populations get small.
        f_res( isnan(f_res) ) = 0;
        f_res_lower( isnan(f_res_lower) ) = 0;
                
        %Data is in (nYears x nCohorts) arrays currently. Rearrange so that it
        %is a linear array with (1 x nYears x nCohorts) elements of increasing
        %time.
        for c = 1:nCohorts
            data_store(c:nCohorts:end,n+1) = f_res(:,c);
            data_output.upperbank_freq(c:nCohorts:end,n+1) = f_res(:,c);            
        end
        
        data_output.lowerbank_freq(:,n+1) = f_res_lower;  
        
%         plot(data_store(:,1),data_store(:,n+1),'.-')
        
        %Use the trapezoid rule to "integrate" the resistance fraction
        %(built in data integration function "trapz")
        metrics.IRF(n) = trapz( data_store(:,1),data_store(:,n+1) ) / ( nCohorts*nYears*time_step );
        
        %% Determine if user defined thresholds were exceeded and at what
        %time
        [metrics.t2R(:,n),counter] = getResistanceTime(data_store(:,1),data_store(:,n+1),f,counter);

        %Update wait bar
        waitbar(n / nSim)
    end
    
    %Delete wait bar upon completion of loop
    delete(h)
    
    %Convert resistance counts to a probability
    metrics.probs = counter / nSim;
        
    %% Compute statistics
    %compute min, 10th,50th,90th percentile, and max time to resistance for
    %each resistance level
    metrics.IRF_stats = mean(metrics.IRF);
    
        for k = 1:length(f)
            %find only simulations where resistance developed
            ind = find(~isinf(metrics.t2R(k,:)));
            tmp_data = metrics.t2R(k,ind);
            
            if ~isempty( tmp_data )
                %min
                metrics.t2R_stats(k,1) = min( tmp_data );

                %10th percentile
                metrics.t2R_stats(k,2) = prctile( tmp_data,10 );

                %50th percentile
                metrics.t2R_stats(k,3) = prctile( tmp_data,50 );

                %90th percentile
                metrics.t2R_stats(k,4) = prctile( tmp_data,90 );

                %max
                metrics.t2R_stats(k,5) = max( tmp_data );
            end

        end
    
    
    
    %% Write data to Excel file
    
    %Get reference to open Excel sheet (still open from parameter entry) 
    h = actxGetRunningServer('Excel.Application');
    myBook = h.Workbooks.Item('WeedResistance_ParameterEntryInterface.xlsm');
    
    %Write metrics to "Results" tab
    mySheet = myBook.Sheets.Item('Results');
    
    %Clear old data (Excel is already doing this, so it is commented out
    %here)
%     mySheet.Range('F7:F13').Value = '';
%     mySheet.Range('C17:D1017').Value = '';
%     mySheet.Range('H8:M1017').Value = '';
%     
%     %Number of simulations
%     mySheet.Range('F7').Value = nSim;
    
    %Probability of exceeding resistances thresholds
    mySheet.Range('F8').Value = metrics.probs(1);
    mySheet.Range('F9').Value = metrics.probs(2);
    mySheet.Range('F10').Value = metrics.probs(3);
    
    %Median times to threshold
    mySheet.Range('F11').Value = metrics.t2R_stats(1,3);
    mySheet.Range('F12').Value = metrics.t2R_stats(2,3);
    mySheet.Range('F13').Value = metrics.t2R_stats(3,3);
    
    %Average integrated risk
    mySheet.Range('F14').Value = mean(metrics.IRF);
    
    %Individual simulation integrated risk
    n = 1:nSim;
    mySheet.Range( strcat('I18:I',num2str(17+nSim)) ).Value = n';
    mySheet.Range( strcat('J18:J',num2str(17+nSim)) ).Value = metrics.IRF;
    
    %Time-to-resistance statistics
    mySheet.Range( strcat('B18:B',num2str(14+length(f))) ).Value = f(4:end);
    mySheet.Range( strcat('C18:G',num2str(14+length(f))) ).Value = metrics.t2R_stats(4:end,1:5);
    
    %Write raw data (upper and lower seed bank resistant fraction) to text
    %file
    dlmwrite('UpperBank.csv',data_output.upperbank_freq,',');
    dlmwrite('LowerBank.csv',data_output.lowerbank_freq,',');
    
end

function [tR,counter] = getResistanceTime(time,res_frac,f,varargin)
    %time is a (Mx1) array. res_frac is an (Mx1) array. time and res_frac
    %must have the same number of rows, otherwise an error is returned. f
    %is the resistant fraction whose time-to-resistance will be found. f
    %may be a (Px1) array. Function returns the time-to-resistance, tR,
    %which is a (Px1) array. varargin is a Px1 array which may be used to
    %keep "count" of how many times resistance developed for each element
    %of f. If resistance is found for current value of f, a counter
    %variable is incremented by 1. This is then returned in counter. This
    %is useful for counting how many times resistance developed over many
    %function calls
    
    if length(time) ~= length(res_frac)
        error('time and res_frac must have equal number of rows');
        
    else
        %Counter variable
        if ~isempty(varargin)
            counter = varargin{1};
        else
            counter = zeros(length(f),1);
        end
        
        %number of evaluation points, P
        P = length(f);
        
        %output variable, a (Px1) array
        tR = zeros(P,1);
        
        %Loop through f and find corresponding times.
        for j = 1:P
            tmp = find(res_frac>f(j),1,'first');

            if tmp == 1
                %The initial resistance exceeded the threshold
                tR(j) = 0;
                
                %Increment counter
                counter(j) = counter(j) + 1;
            elseif ~isempty(tmp)
                %If the threshold is exceeded, interpolate to find the time

                %Temporary time and resistant fraction variables for
                %readability
                t2 = time(tmp);
                t1 = time(tmp-1);
                f2 = res_frac(tmp);
                f1 = res_frac(tmp-1);

                %Linear interpolation
                tR(j) = (t2 - t1) / (f2 - f1) * (f(j) - f1) + t1;
                
                %Increment counter
                counter(j) = counter(j) + 1;
            else
                %The threshold was never exceeded
                tR(j) = Inf;
                


In [ ]:
function runSimulation(varargin)
# % Function simulates the weed life-cycle for specified number of
# % simulations. Function reads parameter input Excel sheet in the same
# % directory. The function writes simulation results to the same
# % spreadsheet, as well as writing csv files containing population data. The
# % user may pass the Params structure from a previous simulation, if
# % desired.

# %% Initialization & preliminary calculations

    # %Create a program status bar to alert user how far along the simulation
    # %is
    h = waitbar(0, 'Please wait', 'Name', 'Plot progress', ...
    'CreateCancelBtn', 'setappdata(gcbf, ''canceling'', 1)');
    setappdata(h, 'canceling', 0)

    # %Run Main function with no arguments once to get the Params variable
    # %populated, unless user has included Params as an argument of the
    # %runSimulation function (in varargin).
    if ~isempty(varargin)
        Params = varargin{1};
    else
        tmp = readData();
        Params = prepareParams_2(tmp);
    end
    
    # %Store the number of simulations desired in local variable. If the
    # %fieldsize is infinite, simulation is deterministic, so nSim is set to
    # %1.
    if isinf(Params.General.fieldSize)
        nSim = 1;
    else
        nSim = Params.General.nSim;
    end
    
    %Store the number of years and cohorts in local variables
    nYears = Params.General.nYears;
    nCohorts = Params.General.nCohorts;
    
    %Frequency values to find the corresponding resistance time for.
    %Includes three user defined values.
    f_RO = Params.General.f_RO;     %resistance onset frequency
    f_CR = Params.General.f_CR;     %critical resistance frequency
    f_TR = Params.General.f_TR;     %total resistance frequency
    f = [f_RO;f_CR;f_TR];           %array containing 3 values
    f_len = length(f);              %length of array
    
    %Counter variable for keeping track of the number of times resistance
    %thresholds are exceeded. 
    counter = zeros(f_len,1);
    
    %Variables to store results
    
    %Variable to locally store the seed bank population
    data_store = zeros(nCohorts*nYears,nSim+1);
    
    # %Variables for sending to the csv files (includes upper and lower seed
    # %banks). Both resistance frequency and total density are stored and
    # %written to files. NOTE: I retroactively added this variable. It
    # %probably would be better code to have data_store and freq_output be
    # %combined into a single array.
    freq_output = struct('upperbank_freq',zeros(nCohorts*nYears,nSim+1),...
                         'lowerbank_freq',zeros(nYears,nSim+1));
    dens_output = struct('upperbank_freq',zeros(nCohorts*nYears,nSim+1),...
                         'lowerbank_freq',zeros(nYears,nSim+1));
    
    # %Make first columns contain time values
    time_step = 1/nCohorts;
    data_store(:,1) = 0:time_step:nYears-time_step;
    freq_output.upperbank_freq(:,1) = 0:time_step:nYears-time_step;
    freq_output.lowerbank_freq(:,1) = 0:nYears-1;
    dens_output.upperbank_freq(:,1) = 0:time_step:nYears-time_step;
    dens_output.lowerbank_freq(:,1) = 0:nYears-1;
    
    # %Structure array for holding population metrics
    metrics = struct('IRF',zeros(nSim,1),'IRF_stats',0,...
                                         'probs',zeros(f_len,1),...
                                         't2R',Inf(f_len,nSim),...
                                         't2R_stats',Inf(f_len,5),...
                                         'yearly_freq',{cell(nYears*nCohorts,6)} );
    

    
%% Perform nSim simulations, storing the results 
    for n = 1:nSim
        
        %Call main function and store Population structure array
        [ Population,~ ] = Main(Params);
       
    %% Compute metrics
        %Sum over all genotypes to find the total resistant and total seed
        %population for each cohort/year in the upper and lower banks
        R = sumResistant( Population.seedBank,Params.key );
        T = squeeze( sum(Population.seedBank,2) );
        R_lower = sumResistant( Population.lowerBank,Params.key );
        T_lower = squeeze( sum(Population.lowerBank,2) );
        
        %Calculate fraction resistant for each year/cohort
        f_res = R./T;
        f_res_lower = (R_lower ./ T_lower);
        
        %0/0 division errors produce NaN for f_res. Replace NaN by
        %0, since NaN errors come when populations approach 0.
        f_res( isnan(f_res) ) = 0;
        f_res_lower( isnan(f_res_lower) ) = 0;
                
        %Data is in (nYears x nCohorts) arrays currently. Rearrange so that it
        %is a linear array with (1 x nYears x nCohorts) elements of increasing
        %time.
        for c = 1:nCohorts
            data_store(c:nCohorts:end,n+1) = f_res(:,c);
            freq_output.upperbank_freq(c:nCohorts:end,n+1) = f_res(:,c);    
            dens_output.upperbank_freq(c:nCohorts:end,n+1) = T(:,c);  
        end
        
        %Store lower bank data
        freq_output.lowerbank_freq(:,n+1) = f_res_lower;  
        dens_output.lowerbank_freq(:,n+1) = T_lower;  
                
        %Use the trapezoid rule to "integrate" the resistance fraction
        %(built in data integration function "trapz")
        metrics.IRF(n) = trapz( data_store(:,1),data_store(:,n+1) ) / ( nCohorts*(nYears-1)*time_step );
        
        %% Determine if user defined thresholds were exceeded and at what
        %time
        [metrics.t2R(:,n),counter] = getResistanceTime(data_store(:,1),data_store(:,n+1),f,counter);

        %Update wait bar
        waitbar(n / nSim)
    end

%% Post-processing of simulation results
    %Delete wait bar upon completion of loop
    delete(h)
    
    %Convert resistance counts to a probability
    metrics.probs = counter / nSim;
        
    %% Compute statistics
    
    %Find the mean of the integrated resistance fraction from each
    %simulation
    metrics.IRF_stats = mean(metrics.IRF);
    
    %Compute min, 10th,50th,90th percentile, and max times required to
    %achieve the user specified resistance thresholds    
        for k = 1:length(f)
            %find only simulations where resistance developed
            ind = find(~isinf(metrics.t2R(k,:)));
            tmp_data = metrics.t2R(k,ind);
            
            if ~isempty( tmp_data )
                %min
                metrics.t2R_stats(k,1) = min( tmp_data );

                %10th percentile
                metrics.t2R_stats(k,2) = prctile( tmp_data,10 );

                %50th percentile
                metrics.t2R_stats(k,3) = prctile( tmp_data,50 );

                %90th percentile
                metrics.t2R_stats(k,4) = prctile( tmp_data,90 );

                %max
                metrics.t2R_stats(k,5) = max( tmp_data );
            end

        end
        
    %Compute the min, 10th, 50th, 90th percentile, and max values of
    %resistance for each year/cohort. If resistance values are all zeros,
    %this cohort/year is ignored (and the value is left blank but not 0).
	metrics.yearly_freq(:,1) = num2cell( data_store(:,1)+1 );
        for y = 1:nCohorts*nYears
            %
            ind = find(data_store(y,2:end)~=0);
            if ~isempty(ind)
                metrics.yearly_freq{y,2} = min(data_store(y,ind+1),[],2);
                metrics.yearly_freq{y,3} = prctile(data_store(y,ind+1),10,2);
                metrics.yearly_freq{y,4} = prctile(data_store(y,ind+1),50,2);
                metrics.yearly_freq{y,5} = prctile(data_store(y,ind+1),90,2);
                metrics.yearly_freq{y,6} = max(data_store(y,ind+1),[],2);
            end
        end
    
    %% Write data to Excel file
    
    %Get reference to open Excel sheet (still open from parameter entry) 
    h = actxGetRunningServer('Excel.Application');
    myBook = h.Workbooks.Item('WeedResistanceGUI.xlsm');
    
    %Write metrics to "Results" tab
    mySheet = myBook.Sheets.Item('Results');
    
    %Clear old data (Excel is already doing this, so it is commented out
    %here)
%     mySheet.Range('F7:F13').Value = '';
%     mySheet.Range('C17:D1017').Value = '';
%     mySheet.Range('H8:M1017').Value = '';
%     
%     %Number of simulations
%     mySheet.Range('F7').Value = nSim;
    
    %Probability of exceeding resistances thresholds
    mySheet.Range('F8').Value = metrics.probs(1);
    mySheet.Range('F9').Value = metrics.probs(2);
    mySheet.Range('F10').Value = metrics.probs(3);
    
    %Median times to threshold (convert to string in case value = Inf)
    mySheet.Range('F11').Value = num2str( metrics.t2R_stats(1,3) );
    mySheet.Range('F12').Value = num2str( metrics.t2R_stats(2,3) );
    mySheet.Range('F13').Value = num2str( metrics.t2R_stats(3,3) );
    
    %Average integrated risk
    mySheet.Range('F14').Value = mean(metrics.IRF);
    
    %Individual simulation integrated risk
    n = 1:nSim;
    mySheet.Range( strcat('I18:I',num2str(17+nSim)) ).Value = n';
    mySheet.Range( strcat('J18:J',num2str(17+nSim)) ).Value = metrics.IRF;
    
    %Resistance versus time statistics
    [rows,~] = size(metrics.yearly_freq);
    mySheet.Range( strcat('B18:B',num2str(17+rows)) ).Value = metrics.yearly_freq(:,1);
    mySheet.Range( strcat('C18:G',num2str(17+rows)) ).Value = metrics.yearly_freq(:,2:end);
    
    %Write raw data (upper and lower seed bank resistant fraction) to text
    %file
    dlmwrite('UpperBankFreq.csv',freq_output.upperbank_freq,',');
    dlmwrite('LowerBankFreq.csv',freq_output.lowerbank_freq,',');
    dlmwrite('UpperBankDens.csv',dens_output.upperbank_freq,',');
    dlmwrite('LowerBankDens.csv',dens_output.lowerbank_freq,',');
    
end

function [tR,counter] = getResistanceTime(time,res_frac,f,varargin)
    %time is a (Mx1) array. res_frac is an (Mx1) array. time and res_frac
    %must have the same number of rows, otherwise an error is returned. f
    %is the resistant fraction whose time-to-resistance will be found. f
    %may be a (Px1) array. Function returns the time-to-resistance, tR,
    %which is a (Px1) array. varargin is a Px1 array which may be used to
    %keep "count" of how many times resistance developed for each element
    %of f. If resistance is found for current value of f, a counter
    %variable is incremented by 1. This is then returned in counter. This
    %is useful for counting how many times resistance developed over many
    %function calls
    
    if length(time) ~= length(res_frac)
        error('time and res_frac must have equal number of rows');
        
    else
        %Counter variable
        if ~isempty(varargin)
            counter = varargin{1};
        else
            counter = zeros(length(f),1);
        end
        
        %number of evaluation points, P
        P = length(f);
        
        %output variable, a (Px1) array
        tR = zeros(P,1);
        
        %Loop through f and find corresponding times.
        for j = 1:P
            tmp = find(res_frac>f(j),1,'first');

            if tmp == 1
                %The initial resistance exceeded the threshold
                tR(j) = 0;
                
                %Increment counter
                counter(j) = counter(j) + 1;
            elseif ~isempty(tmp)
                %If the threshold is exceeded, interpolate to find the time

                %Temporary time and resistant fraction variables for
                %readability
                t2 = time(tmp);
                t1 = time(tmp-1);
                f2 = res_frac(tmp);
                f1 = res_frac(tmp-1);

                %Linear interpolation
                tR(j) = (t2 - t1) / (f2 - f1) * (f(j) - f1) + t1;
                
                %Increment counter
                counter(j) = counter(j) + 1;
            else
                %The threshold was never exceeded
                tR(j) = Inf;
                


In [ ]:
function [ ] = samplePlots( )
%% Initialization

P = readData();

%create figure
figure('Position',get(0,'ScreenSize'))

%choose lifestage to plot
lifestage = 'seedBank';

h = waitbar(0, 'Please wait', 'Name', 'Plot progress', ...
    'CreateCancelBtn', 'setappdata(gcbf, ''canceling'', 1)');
setappdata(h, 'canceling', 0)

%run simulation number of times specified and plot emergence curve for each
subplot(2,2,1)
hold all
nSim = P.General.nSim;      %number of simulations to run
nRes = 0;       %number of times resistance develops after nYears


%% Run simulation specified number of times 

%Resistant fraction evaulation points
fEval = 0.05:0.05:0.95;
yearEval = zeros(nSim,length(fEval));

%Default threshold for when population is considered resistant
toler = P.General.criticalResTime;

%Create array to store resistance metric data for all simulations
resData = zeros(nSim, 3);

%Run simulation
for n = 1:nSim

    if getappdata(h, 'canceling')
        break
    end
    
    [Population,Params] = Main_2();
    plotPopulation(Population.(lifestage),Params.key,'fracR');
    
    %Compute resistance risk: fraction of times resistant population is >= 25% of
    %total after N years, (defined by nYears in defineParams() function)
    R = sumResistant(Population.(lifestage)(:,:,end),Params.key); %resistant population
    T = sum(Population.(lifestage)(:,:,end),2);    %total population (at end of year)
    
    if R(end)/T(end) >= 0.25
        nRes = nRes + 1;
    end

    %Create variable for storing the resistance risk using the Population
    %parameter (first time through loop only)
    if n==1
        ResistanceRisk = zeros(1,Params.General.nYears);
    end

    %count the resistants in each year and divide by total population
    resPop = sumResistant(Population.(lifestage)(:,:,end),Params.key);
    totPop = sum(Population.(lifestage)(:,:,end),2);
    resFrac = resPop./totPop;

    %find the years when the resistant fraction exceeds the tolerance
    ind = find(resFrac>=toler);
    ResistanceRisk(ind) = ResistanceRisk(ind) + 1;

    %find fractional year values corresponding to the
    %resistant_fraction array specified by the user
    yearEval(n,:) = sigmoidMetrics(0:length(resFrac)-1,resFrac,fEval);

    %Store resistance metric data for this simulation
    resData(n, :) = Population.Res;
    
    %Update the progress bar
    waitbar(n / nSim)
end

delete(h)

%% Store resistance metric data in Excel

range = strcat('D17:F', num2str(17+nSim-1));
h = actxGetRunningServer('Excel.Application');
myBook = h.Workbooks.Item('WeedResistance_ParameterEntryInterface.xlsm');
mySheet = myBook.Sheets.Item('Results');
mySheet.Range(range).Value = resData;

%% Add title and resistance risk text to first plot
title(strcat('Fraction resistant: ',num2str(nSim), ' simulations'))
pos = Params.General.nYears / 2;
text(pos,0.2,strcat('Resistance risk = ',num2str(nRes/nSim)));

%% Plots with +- one standard deviation window

%convert ResistanceRisk to a probability
ResistanceRisk = ResistanceRisk/nSim;

%eliminate curves where didn't or partially developed
k = 1;
while k <= size(yearEval,1)
    if sum(isnan(yearEval(k,:)))~=0
        yearEval(k,:) = [];
    else
        k = k + 1;
    end
end

%Take average, standard deviation, and store resistant_fraction in array
%meanCurve, for output
meanCurve(:,1) = mean(yearEval, 1);
meanCurve(:,2) = std(yearEval, 0, 1);
meanCurve(:,3) = fEval;

%Fit a sigmoid to the output (using subfunction sigmoidFun, below)
p = lsqcurvefit(@sigmoidFun,[0.5,5],meanCurve(:,1),meanCurve(:,3));

%Use fit parameters to create a new population variable pop, which may
%be smoother than the original simulation data.
t = linspace(0,Params.General.nYears-1);
pop = sigmoidFun(p,t);
t = t';
pop = pop';

% ax1 = axes('Position',[0.1,0.1,0.38,0.85]);
subplot(2,2,2)
hold all
% set(0,'DefaultAxesColorOrder',[0 0 1;1 0 0;0 0.4 0;1 0.6 0])

%lower window
p = lsqcurvefit(@sigmoidFun,[0.5,5],meanCurve(:,1)-2*meanCurve(:,2),meanCurve(:,3));
wind = sigmoidFun(p,t);
area(t,wind,'FaceColor',[0.7,0.7,0.7],'LineStyle','none')
plot(t,wind,'k','LineWidth',1,'LineStyle',':')

%upper window
p = lsqcurvefit(@sigmoidFun,[0.5,5],meanCurve(:,1)+2*meanCurve(:,2),meanCurve(:,3));
wind = sigmoidFun(p,t);
area(t,wind,'FaceColor',[1,1,1],'LineStyle','none')
plot(t,wind,'k','LineWidth',1,'LineStyle',':')

plot(t,pop,'k-','LineWidth',3)
%plot(t,-2*wind,'LineWidth',3,'LineStyle',':')
xlabel('Year','Interpreter','latex')
ylabel('Fraction resistant','Interpreter','latex')
ax = gca;
ax.FontSize = 16;

%% Resistance risk versus time plot

%Make data look smooth using Piecewise Cubic Hermite Interpolating
%Polynomial (PCHIP)

%Years in original simulation
n = 0:length(ResistanceRisk)-1;

%Points to do the PCHIP through
t = linspace(0,max(n));

%Call pchip
pop = pchip(n,ResistanceRisk,t);

%Plot results
t = t';
pop = pop';

% ax2 = axes('Position',[0.6,0.1,0.38,0.85]);
subplot(2,2,3)
hold all
plot(t,pop*100,'k-','LineWidth',3,'MarkerSize',20)
xlabel('Year','Interpreter','latex')
ylabel('Resistance risk $\%$','Interpreter','latex')
ax = gca;
ax.FontSize = 16;

end

function y = sigmoidFun(x,xdata)
    y = 1 ./ (1 + exp(-x(1).*xdata + x(2)));
end

function Rpop = sumResistant(pop,key)

    %Determine number of gene loci and genotypes from key variable
    [nLoci,nGeno] = size(key);
    
    %Determine number of years and cohorts from population variable
    [nYears,~,nCohorts] = size(pop);
    
    %Initialize output array to 0
    Rpop = zeros(nYears,nCohorts);
    
    %Sum all with resistant allele
    for G = 1:nGeno
        for L = 1:nLoci
            if key(L,G) == 2 || key(L,G) == 1
                Rpop(:,:) = Rpop(:,:) + squeeze(pop(:,G,:));
                break
            end
        end
    end
    
end

function nEval = sigmoidMetrics(n,f,fEval)
    
    %Number of evaluation points
    len = length(fEval);

    %Array to store output
    nEval = zeros(1,len);
    
    %Loop through evaluation points and populate nEval
    for k = 1:len
        
        %Find the first value >= current evaluation point
        ind = find(f>=fEval(k),1,'first');
        
        %If no value found or if the population initially exceeds
        %tolerance, store NaN, otherwise, find the time corresponding to
        %the current evaulation point (using linear interpolation)
        if isempty(ind)
            nEval(k) = NaN;
        elseif ind == 1
            nEval(k) = NaN;
        else
            nEval(k) = (n(ind)-n(ind-1))/(f(ind)-f(ind-1))*(fEval(k)-f(ind-1)) + n(ind-1);



In [ ]:
function seedIn = seedImmigration(nGeno,varargin)
    seedIn = zeros(1,nGeno);

In [ ]:
function newPop = selectiveSurvival(oldPop,fSurvival,targetLocus,key)
%fSurvival = 3 x 1 array containing: [fSS,fRS,fRR]
[nLoci,nGeno] = size(key);
newPop = oldPop*0;

if targetLocus > nLoci
    warning('Target locus exceeds the number specified by the user in the input file, original population returned');
    newPop = oldPop;
else
   for k = 1:nGeno
       switch key(targetLocus,k)
           case 0
               %homozygous susceptible
                newPop(k) = fSurvival(1)*oldPop(k);
           case 1
               %heterozygous
               newPop(k) = fSurvival(2)*oldPop(k);
           case 2
               newPop(k) = fSurvival(3)*oldPop(k);
       end
       
   end
   
end

In [ ]:
function nEval = sigmoidMetrics(n,f,fEval)
    
    %Number of evaluation points
    len = length(fEval);

    %Array to store output
    nEval = zeros(1,len);
    
    %Loop through evaluation points and populate nEval
    for k = 1:len
        
        %Find the first value >= current evaluation point
        ind = find(f>=fEval(k),1,'first');
        
        %If no value found or if the population initially exceeds
        %tolerance, store NaN, otherwise, find the time corresponding to
        %the current evaulation point (using linear interpolation)
        if isempty(ind)
            nEval(k) = NaN;
        elseif ind == 1
            nEval(k) = NaN;
        else
            nEval(k) = (n(ind)-n(ind-1))/(f(ind)-f(ind-1))*(fEval(k)-f(ind-1)) + n(ind-1);
        end
        
    end
end

In [ ]:
function Rpop = sumResistant(pop,key,varargin)

if isempty(varargin)
    sumOver = 'all';
else
    sumOver = varargin{1};
end

%determine number of gene loci and genotypes from key variable
[nLoci,nGeno] = size(key);

%determine number of years and cohorts from population variable
[nYears,~,nCohorts] = size(pop);

%initialize output array to 0
Rpop = zeros(nYears,nCohorts);

switch sumOver

    case 'homozygous'
        for G = 1:nGeno
            for L = 1:nLoci
                if key(L,G) == 2 
                    Rpop(:,:) = Rpop(:,:) + squeeze(pop(:,G,:));
                    break
                end
            end
        end
        
    otherwise
        for G = 1:nGeno
            for L = 1:nLoci
                if key(L,G) == 2 || key(L,G) == 1
                    Rpop(:,:) = Rpop(:,:) + squeeze(pop(:,G,:));
                    break
                end
            end
        end
        
end

In [ ]:
function newPopulation = survival(oldPopulation,fSurvival)
    newPopulation = fSurvival*oldPopulation;

In [14]:
# creating a variable and storing the text
# that we want to search
def search_replace(filename,search, replace):
	search_text = search

	# creating a variable and storing the text
	# that we want to add
	replace_text = replace

	# Opening our text file in read only
	# mode using the open() function
	with open(filename, 'r') as file:

		# Reading the content of the file
		# using the read() function and storing
		# them in a new variable
		data = file.read()

		# Searching and replacing the text
		# using the replace() function
		data = data.replace(search_text, replace_text)

	# Opening our text file in write only
	# mode to write the replaced content
	filename = filename.split('.')[0]+'.py'
	with open(filename, 'w') as file:

		# Writing the replaced data in our
		# text file
		file.write(data)

	# Printing Text replaced
	print("Text replaced")


In [20]:
import os
for i in os.listdir('BMP_Matlab_Code/'):
    if i.endswith('.py'):
        print(i)
        search_replace('BMP_Matlab_Code/'+i,'zeros','np.zeros')

competition.py
Text replaced
deepTill.py
Text replaced
extinction.py
Text replaced
getResistanceTime.py
Text replaced
Herbicide_Data.py
Text replaced
LowerBankDens.py
Text replaced
LowerBankFreq.py
Text replaced
Main.py
Text replaced
MainSimulation.py
Text replaced
mating.py
Text replaced
mutation.py
Text replaced
PackagingLog.py
Text replaced
plotPopulation.py
Text replaced
prepareParams_2.py
Text replaced
readData.py
Text replaced
resistanceRisk.py
Text replaced
resMetrics.py
Text replaced
runSimulation.py
Text replaced
runSimulation_old_working.py
Text replaced
samplePlots.py
Text replaced
seedImmigration.py
Text replaced
selectiveSurvival.py
Text replaced
selfTable1.py
Text replaced
selfTable2.py
Text replaced
selfTable3.py
Text replaced
selfTable4.py
Text replaced
selfTable5.py
Text replaced
sigmoidMetrics.py
Text replaced
sumResistant.py
Text replaced
survival.py
Text replaced
test.py
Text replaced
